## Exploratory Data Analysis
Week 2 submission

Contents:

- UPDATE

In [2]:
#import relevant tools
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold



In [ ]:
#get data
data = pd.read_csv("data/core/systems_cleaned.csv")
#focus only on the locations that have irradiance data
data_irrad = data.loc[data["has_irrad_data"]]
data_irrad
#note that only 5 of these are from the prize_data, so they might be the only once with enough irradiance data?

,system_id,system_public_name,site_location,timezone_or_utc_offset,latitude,longitude,elevation_m,dc_capacity_kW,kg_climate,pvcz_composite,...,number_records,dataset_size_mb,available_sensor_channels,qa_status,qa_issue,first_year,is_prize_data,is_lake_parquet_data,is_lake_csv_data,has_irrad_data
0,2,Residential 1a,"Lakewood, CO",America/Denver,39.7214,-105.0972,1675.0,2.912,Dfb,12,...,1.368590e+07,313.25,7,fail,less than 1.0 years data,2010.0,False,True,False,True
1,3,Residential 1b,"Lakewood, CO",America/Denver,39.7214,-105.0972,1675.0,2.720,Dfb,12,...,1.266818e+07,289.95,7,fail,NaN,2010.0,False,True,False,True
2,4,NREL x-Si -1,"Golden, CO",7,39.7406,-105.1774,1795.3,1.000,BSk,12,...,1.139780e+08,2608.75,15,pass,"Filtered time series less than 1.0 years data,...",2007.0,False,True,False,True
3,10,NREL CIS -1,"Golden, CO",7,39.7404,-105.1774,1792.8,1.120,BSk,12,...,1.131036e+08,2588.74,14,pass,Filtered time series less than 1.0 years data,2006.0,False,True,False,True
4,33,Silicor Materials,"Golden, CO",7,39.7404,-105.1772,1794.0,2.400,BSk,12,...,1.136736e+08,2601.78,15,pass,"Percent clipping exceeded threshold of 10%, Fi...",2010.0,False,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,4902,NIST_Ground_1,Gaithersburg MD,5,39.1319,-77.2141,138.0,270.700,Cfa,34,...,1.823688e+08,4174.09,98,pass,"Percent clipping exceeded threshold of 10%, Fi...",2014.0,False,True,False,True
159,4903,NIST_Roof_1,Gaithersburg MD,5,39.1354,-77.2156,149.0,73.700,Cfa,34,...,2.436256e+08,5576.15,121,pass,"Filtered time series less than 1.0 years data,...",2014.0,False,True,False,True
160,7333,Shine On Solar Facility - 10sec,"Levis, CA",8,36.5780,-120.4016,330.1,205300.000,Bsk,34,...,4.064245e+10,930231.96,4086,fail,NaN,2016.0,True,False,False,True
162,9068,SR_CO,"Kersey, CO",7,40.3864,-104.5512,1407.0,4738.000,Bsk,12,...,3.296997e+08,7546.23,438,pass,"Filtered time series less than 1.0 years data,...",2017.0,True,False,False,True


In [15]:
#figure out where all these things are located
import plotly.express as px
fig = px.density_map(data_irrad, 
                     lat='latitude', 
                     lon='longitude', 
                     z='elevation_m', #ideally irradiance or degradation
                     radius=20,
                    center=dict(lat=40, lon=-98), 
                    zoom=2.8,
                    map_style="open-street-map")
fig.show()

In [ ]:
#plot variables against each other
#first, figure out what the variables even are
data_irrad.columns

Index(['system_id', 'system_public_name', 'site_location',
       'timezone_or_utc_offset', 'latitude', 'longitude', 'elevation_m',
       'dc_capacity_kW', 'kg_climate', 'pvcz_composite', 'pvcz_t_rack',
       'pvcz_t_roof', 'pvcz_humidity', 'pvcz_wind', 'tracking', 'type',
       'azimuth', 'tilt', 'first_timestamp', 'last_timestamp', 'years',
       'number_records', 'dataset_size_mb', 'available_sensor_channels',
       'qa_status', 'qa_issue', 'first_year', 'is_prize_data',
       'is_lake_parquet_data', 'is_lake_csv_data', 'has_irrad_data'],
      dtype='str')

In [27]:
#what are the types in each column?
data_irrad = data.loc[data["has_irrad_data"]]
data_irrad=data_irrad.drop(columns=["is_prize_data",
                                    "is_lake_parquet_data",
                                    "is_lake_csv_data",
                                    "has_irrad_data",
                                    "timezone_or_utc_offset",
                                    "first_timestamp",
                                    "last_timestamp",
                                    "system_public_name",
                                    "first_year"])
data_irrad.info()

<class 'pandas.DataFrame'>
Index: 66 entries, 0 to 163
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   system_id                  66 non-null     int64  
 1   site_location              66 non-null     str    
 2   latitude                   66 non-null     float64
 3   longitude                  66 non-null     float64
 4   elevation_m                66 non-null     float64
 5   dc_capacity_kW             47 non-null     float64
 6   kg_climate                 66 non-null     str    
 7   pvcz_composite             66 non-null     int64  
 8   pvcz_t_rack                66 non-null     int64  
 9   pvcz_t_roof                66 non-null     int64  
 10  pvcz_humidity              66 non-null     int64  
 11  pvcz_wind                  66 non-null     int64  
 12  tracking                   65 non-null     str    
 13  type                       65 non-null     str    
 14  azimuth    